# Chapter 09: Instruction Tuning

## 1. 학습 목표

* Instruction Tuning의 개념과 Base Model과의 차이를 이해한다.
* Alpaca, ChatML, Llama 등 다양한 Instruction 포맷을 구현한다.
* 여러 데이터셋(Dolly, Alpaca, Orca 등)을 혼합하여 데이터 다양성을 확보한다.
* 효과적인 Instruction 데이터 구성 전략을 수립한다.

## 2. Instruction Tuning이란?

**Base Model**은 단순히 "다음에 올 단어를 예측"하도록 학습되었다. 따라서 질문을 하면 답변을 하는 대신 질문을 연이어 생성하거나, 엉뚱한 텍스트를 이어 붙이기도 한다.

**Instruction Tuning**은 이러한 모델에게 **"지시(Instruction)와 입력(Input)이 주어지면 적절한 응답(Response)을 해야 한다"**는 패턴을 학습시키는 과정이다.

```text
Base Model:
입력: "파리는"
출력: "프랑스의 수도이며..." (단순 완성)

Instruction Tuned Model:
입력: "프랑스의 수도는 어디인가?"
출력: "프랑스의 수도는 파리입니다." (질문 의도 파악 및 답변)

```

## 3. 다양한 Instruction 포맷

모델마다, 또는 데이터셋마다 선호하는 프롬프트 형식이 다르다. 대표적인 세 가지 포맷을 코드로 정의해본다.

In [1]:
# 1. Alpaca 스타일 (가장 보편적)
# 지시사항, 입력(선택), 응답으로 명확히 구분된다.
alpaca_format = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

# 2. ChatML 스타일 (OpenAI, Qwen 등)
# 시스템, 사용자, 어시스턴트 역할이 태그로 구분된다.
chatml_format = """<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{instruction}<|im_end|>
<|im_start|>assistant
{output}<|im_end|>"""

# 3. Llama-2/3 스타일
# [INST] 태그를 사용하여 지시사항을 감싼다.
llama_format = """[INST] {instruction} [/INST] {output}"""

print("지원하는 포맷 정의 완료: Alpaca, ChatML, Llama")

지원하는 포맷 정의 완료: Alpaca, ChatML, Llama


## 4. 데이터셋 혼합 (Data Mixing) 전략

하나의 데이터셋만 사용하면 모델이 해당 데이터의 특정 패턴(말투, 길이 등)에 과적합될 수 있다. 성격이 다른 여러 데이터셋을 혼합하여 **일반화 능력**을 높이는 것이 중요하다.

### 4.1 데이터셋 로드 및 혼합

In [2]:
from datasets import load_dataset, concatenate_datasets
import numpy as np

def prepare_mixed_dataset():
    """
    여러 오픈소스 데이터셋을 로드하고 비율에 맞춰 혼합하는 함수다.
    """
    print("데이터셋 혼합 시작...")

    # 사용할 데이터셋 목록과 비율
    # 1. Dolly: 창의적 작문, 브레인스토밍에 강점
    # 2. Alpaca: 간결한 지시 이행에 강점
    dataset_configs = [
        ("nlpai-lab/databricks-dolly-15k-ko", 0.5, "train"),
        ("tatsu-lab/alpaca", 0.5, "train")
    ]

    mixed_data = []

    for name, ratio, split in dataset_configs:
        print(f"로드 중: {name} ({ratio*100}%)")
        # 실습용으로 1000개만 로드하지만, 실제로는 전체 데이터를 사용한다.
        ds = load_dataset(name, split=f"{split}[:1000]")

        # 데이터셋마다 컬럼명이 다를 수 있어 통일 작업이 필요하다.
        # 여기서는 간단히 필요한 컬럼만 매핑한다고 가정한다.
        # 실제로는 rename_column 등을 사용해야 한다.

        mixed_data.append(ds)

    # 데이터셋 병합
    combined_dataset = concatenate_datasets(mixed_data)

    # 셔플 (데이터 순서 섞기)
    shuffled_dataset = combined_dataset.shuffle(seed=42)

    print(f"혼합 완료: 총 {len(shuffled_dataset)}개 데이터")
    return shuffled_dataset

# 데이터셋 준비
mixed_dataset = prepare_mixed_dataset()

데이터셋 혼합 시작...
로드 중: nlpai-lab/databricks-dolly-15k-ko (50.0%)
로드 중: tatsu-lab/alpaca (50.0%)
혼합 완료: 총 2000개 데이터


## 5. 프롬프트 생성 함수 구현

다양한 소스의 데이터를 하나의 통일된 학습 포맷(여기서는 Alpaca 스타일)으로 변환하는 함수를 구현한다.

In [3]:
def create_instruction_prompt(example):
    """
    데이터 샘플을 받아 학습용 프롬프트 텍스트를 생성하는 함수다.
    Alpaca 포맷을 기본으로 사용한다.
    """
    # 데이터셋마다 필드명이 다를 수 있으므로 get으로 안전하게 가져온다.
    # Dolly 데이터셋의 경우 'context'가 'input' 역할을 한다.
    instruction = example.get('instruction', '')
    input_text = example.get('input', example.get('context', ''))
    output = example.get('output', example.get('response', ''))

    # 내용이 없는 경우 건너뛰거나 빈 문자열 처리
    if not instruction or not output:
        return {"text": ""}

    # 포맷팅
    if input_text:
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### Instruction:
{instruction}

### Response:
{output}"""

    # EOS 토큰 추가 (모델이 생성을 멈추는 지점을 학습)
    text += "<|endoftext|>"

    return {"text": text}

# 변환 적용
formatted_mixed_dataset = mixed_dataset.map(create_instruction_prompt)

# 결과 확인
print("\n[변환된 데이터 샘플]")
print(formatted_mixed_dataset[0]['text'])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]


[변환된 데이터 샘플]
### Instruction:
Classify this sentence as either a declarative or interrogative

### Input:
Do you agree

### Response:
Interrogative.<|endoftext|>


## 6. 효과적인 데이터 구성 전략

단순히 양만 늘리는 것이 아니라, 태스크의 **다양성(Diversity)**을 확보하는 것이 Instruction Tuning의 핵심이다. 다음과 같은 유형의 데이터가 골고루 포함되어야 한다.

1. **설명 (Explanation)**: 개념이나 현상을 설명 ("양자역학이 뭐야?")
2. **생성 (Generation)**: 시, 소설, 코드 작성 ("피보나치 수열 파이썬 코드 짜줘")
3. **요약 (Summarization)**: 긴 글을 짧게 요약
4. **추론 (Reasoning)**: 단계별 사고가 필요한 문제 ("A가 B보다 크고 B가 C보다 작을 때...")
5. **분류 (Classification)**: 텍스트의 감정 분석이나 카테고리 분류

In [4]:
# 데이터 다양성 예시 출력
task_examples = {
    "설명": "머신러닝과 딥러닝의 차이를 설명하라.",
    "생성": "가을을 주제로 한 하이쿠를 지어라.",
    "요약": "다음 뉴스 기사를 3문장으로 요약하라.",
    "코딩": "React로 간단한 투두 리스트 컴포넌트를 작성하라."
}

print("권장되는 데이터 다양성 예시:")
for task, example in task_examples.items():
    print(f"- [{task}]: {example}")

권장되는 데이터 다양성 예시:
- [설명]: 머신러닝과 딥러닝의 차이를 설명하라.
- [생성]: 가을을 주제로 한 하이쿠를 지어라.
- [요약]: 다음 뉴스 기사를 3문장으로 요약하라.
- [코딩]: React로 간단한 투두 리스트 컴포넌트를 작성하라.


## 7. 요약

이 챕터에서는 **Instruction Tuning**을 위해 데이터를 준비하고 가공하는 방법을 학습했다.

1. **포맷의 중요성**: Alpaca, ChatML 등 모델에 맞는 프롬프트 형식을 사용해야 성능이 극대화된다.
2. **데이터 믹싱**: Dolly(창의성), Alpaca(지시 이행) 등 성격이 다른 데이터를 섞어 일반화 성능을 높였다.
3. **다양성 확보**: 특정 유형의 질문에만 과적합되지 않도록 작업 유형을 다양하게 구성해야 한다.

이제 모델이 지시를 잘 따르게 되었다면, 다음 단계는 모델이 **인간의 선호도**에 맞는 '좋은' 답변을 하도록 만드는 것이다.

다음 챕터는 **Chapter 10: DPO (Direct Preference Optimization) Training**으로, RLHF보다 효율적인 방식으로 모델을 정렬(Alignment)하는 방법을 다룬다.